# 1. Modelos

In [3]:
import os
import cv2
from ultralytics import YOLO

In [5]:
def run_yolo_inference(model_path, input_dir, output_dir, conf_threshold=0.25):
    os.makedirs(output_dir, exist_ok=True)

    model = YOLO(model_path)
    image_files = [f for f in os.listdir(input_dir) if f.lower().endswith(('.jpg', '.jpeg', '.png', '.bmp'))]

    for img_name in image_files:
        input_path = os.path.join(input_dir, img_name)
        output_path = os.path.join(output_dir, img_name)

        img = cv2.imread(input_path)
        if img is None:
            print(f"Failed to read {input_path}")
            continue

        results = model(img)[0]

        for box in results.boxes:
            cls_id = int(box.cls[0])
            conf = float(box.conf[0])
            if conf < conf_threshold:
                continue
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            label = f"{model.names[cls_id]} {conf:.2f}"
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.putText(img, label, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        cv2.imwrite(output_path, img)

In [6]:
model_path = "weights/carmodel/best.pt"
input_dir = "O4_test_images"
output_dir = "O4_test_images_results"
run_yolo_inference(model_path, input_dir, output_dir)



0: 640x640 1 DFSK_Glory_330, 65.0ms
0: 640x640 1 DFSK_Glory_330, 65.0ms
Speed: 1.3ms preprocess, 65.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)
Speed: 1.3ms preprocess, 65.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 (no detections), 50.5ms
0: 640x640 (no detections), 50.5ms
Speed: 1.2ms preprocess, 50.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
Speed: 1.2ms preprocess, 50.5ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 (no detections), 50.2ms
0: 640x640 (no detections), 50.2ms
Speed: 1.2ms preprocess, 50.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)
Speed: 1.2ms preprocess, 50.2ms inference, 0.3ms postprocess per image at shape (1, 3, 640, 640)


0: 640x640 1 Hyundai_i20, 47.8ms
0: 640x640 1 Hyundai_i20, 47.8ms
Speed: 1.4ms preprocess, 47.8ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)
Speed: 1.4ms preprocess, 47.8ms inf

In [7]:
model = YOLO(model_path)

In [9]:
print(model.names)

{0: 'Changan_CS35', 1: 'Changan_CS55', 2: 'Changan_New_Van', 3: 'Chevrolet_Camaro', 4: 'Chevrolet_Cruze', 5: 'Chevrolet_Onix', 6: 'Chevrolet_Spark', 7: 'Chevrolet_Tracker', 8: 'DFSK_Glory_330', 9: 'DFSK_Glory_500', 10: 'DFSK_Glory_580', 11: 'Hyundai_Accent', 12: 'Hyundai_Creta', 13: 'Hyundai_Elantra', 14: 'Hyundai_i20', 15: 'Hyundai_Santa_Fe', 16: 'Hyundai_Sonata', 17: 'Hyundai_Tucson', 18: 'Hyundai_Veloster', 19: 'JAC_JS4', 20: 'JAC_T8', 21: 'Kia_Niro', 22: 'Kia_Rio', 23: 'Kia_Seltos', 24: 'Kia_Sorento', 25: 'Kia_Soul', 26: 'Kia_Sportage', 27: 'Nissan_Kicks', 28: 'Nissan_Sentra', 29: 'Nissan_Versa', 30: 'Suzuki_Jimny', 31: 'Suzuki_Swift', 32: 'Suzuki_Vitara', 33: 'Toyota_4Runner', 34: 'Toyota_C-HR', 35: 'Toyota_Camry', 36: 'Toyota_Celica', 37: 'Toyota_Corolla', 38: 'Toyota_Corolla_Cross', 39: 'Toyota_FJ_Cruiser', 40: 'Toyota_Hilux', 41: 'Toyota_Land_Cruiser', 42: 'Toyota_Prius', 43: 'Toyota_RAV4', 44: 'Toyota_Rush', 45: 'Toyota_Supra', 46: 'Toyota_Tercel', 47: 'Toyota_Yaris', 48: 'Vol

In [11]:
import os
import cv2

In [12]:
def draw_yolo_bboxes(image_dir, label_dir, output_dir, class_dict, image_ext=".jpg"):
    os.makedirs(output_dir, exist_ok=True)
    for filename in os.listdir(image_dir):
        if not filename.endswith(image_ext):
            continue
        image_path = os.path.join(image_dir, filename)
        label_path = os.path.join(label_dir, os.path.splitext(filename)[0] + ".txt")
        output_path = os.path.join(output_dir, filename)

        image = cv2.imread(image_path)
        if image is None or not os.path.exists(label_path):
            continue

        h, w = image.shape[:2]
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                cls_id, cx, cy, bw, bh = map(float, parts)
                cls_name = class_dict.get(int(cls_id), str(cls_id))

                x1 = int((cx - bw / 2) * w)
                y1 = int((cy - bh / 2) * h)
                x2 = int((cx + bw / 2) * w)
                y2 = int((cy + bh / 2) * h)

                cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
                cv2.putText(image, cls_name, (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)

        cv2.imwrite(output_path, image)

In [13]:
class_dict = model.names
draw_yolo_bboxes("O4_test_images/", "O4_test_labels/", "O4_test_labels_results/", class_dict)

In [14]:
data_path = 'data_custom_car_model.yaml'
imgsz = 640

In [18]:
results = model.val(
    data=data_path,
    split='test',
    imgsz=imgsz
)

Ultralytics 8.3.114  Python-3.13.3 torch-2.7.0+cpu CPU (AMD Ryzen 9 7900X 12-Core Processor)
Ultralytics 8.3.114  Python-3.13.3 torch-2.7.0+cpu CPU (AMD Ryzen 9 7900X 12-Core Processor)
val: Fast image access  (ping: 0.00.0 ms, read: 1210.4991.0 MB/s, size: 243.9 KB)
val: Fast image access  (ping: 0.00.0 ms, read: 1210.4991.0 MB/s, size: 243.9 KB)


val: Scanning E:\Github\Tesis\O4-Integracion\O4_test\labels... 29 images, 0 backgrounds, 0 corrupt: 100%|██████████| 29/29 [00:00<00:00, 160.72it/s]


val: New cache created: E:\Github\Tesis\O4-Integracion\O4_test\labels.cache
val: New cache created: E:\Github\Tesis\O4-Integracion\O4_test\labels.cache


e:\Github\Tesis\venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.15s/it]


                   all         29         30      0.495      0.297      0.309      0.271
                   all         29         30      0.495      0.297      0.309      0.271
          Changan_CS35          1          1          0          0          0          0
          Changan_CS35          1          1          0          0          0          0
        Hyundai_Accent          5          5      0.503        0.2      0.253      0.152
        Hyundai_Accent          5          5      0.503        0.2      0.253      0.152
       Hyundai_Elantra          2          2          1          0          0          0
       Hyundai_Elantra          2          2          1          0          0          0
      Hyundai_Santa_Fe          1          1      0.196          1      0.249      0.249
      Hyundai_Santa_Fe          1          1      0.196          1      0.249      0.249
        Hyundai_Tucson          1          1          0          0          0          0
        Hyundai_Tucso

# 2. Histograma de F1 Score

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

def save_f1_histogram_from_excel(excel_path, output_img_path):
    df = pd.read_excel(excel_path)
    df = df[df["Class"] != "all"]
    df["F1"] = pd.to_numeric(df["F1"], errors='coerce')
    df = df.dropna(subset=["F1"])

    bins = np.arange(0.0, 1.1, 0.1)

    plt.figure(figsize=(10, 6))
    counts, edges, bars = plt.hist(df["F1"], bins=bins, edgecolor='black', align='mid')

    plt.xticks(bins)
    plt.xlim(0.0, 1.0)  # Fuerza los límites del eje X

    plt.xlabel("F1 Score")
    plt.ylabel("Frecuencia")
    plt.title("Histograma de F1 Score por clase")

    # Etiquetas de frecuencia sobre cada barra
    for count, edge in zip(counts, edges[:-1]):
        center = edge + 0.05  # centro del bin
        if count > 0:
            plt.text(center, count, int(count), ha='center', va='bottom', fontsize=10)

    plt.tight_layout()
    plt.savefig(output_img_path)
    plt.close()


In [6]:
output_csv_path = "O4_models/class_metrics_f1.xlsx"
output_img_path = "O4_models/f1_histogram.jpg"
save_f1_histogram_from_excel(output_csv_path, output_img_path)